In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import time
import numpy as np
import pandas as pd

from tqdm import tqdm_notebook
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
import scipy.optimize
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor, LGBMClassifier
import lightgbm as lgb
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

In [ ]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/2020_蝦皮BestCoder競賽(1121)")

## Read Data

In [ ]:
purchase_detail = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/2020_蝦皮BestCoder競賽(1121)/iamthebestcoderopen2020/purchase_detail.csv", index_col=None)
login = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/2020_蝦皮BestCoder競賽(1121)/iamthebestcoderopen2020/login.csv", index_col=None)
user_info = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/2020_蝦皮BestCoder競賽(1121)/iamthebestcoderopen2020/user_info.csv", index_col=None)
user_label_train = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/2020_蝦皮BestCoder競賽(1121)/iamthebestcoderopen2020/user_label_train.csv", index_col=None)

## Merge Data

### loging資料前處理

In [ ]:
login["month"] = login["date"].str[5:7]
login.head()

,userid,date,login_times,month
0,161097,2020-07-29,2,07
1,243570,2020-07-29,2,07
2,355497,2020-07-29,4,07
3,167925,2020-07-29,1,07
4,504766,2020-07-29,1,07


In [ ]:
login_group = login[["userid", "month", "login_times"]].groupby(["userid", "month"], as_index=False).sum()
login_group.head()

,userid,month,login_times
0,1,02,7
1,1,03,7
2,1,04,6
3,1,05,9
4,1,06,6


In [ ]:
login_pivot = pd.pivot_table(login_group, values='login_times', index=['userid'], columns=['month'], aggfunc=np.sum)
login_pivot

month,02,03,04,05,06,07
userid,,,,,,
1,7.0,7.0,6.0,9.0,6.0,15.0
2,86.0,82.0,71.0,84.0,84.0,80.0
3,130.0,133.0,118.0,122.0,126.0,141.0
4,85.0,68.0,65.0,66.0,71.0,71.0
5,151.0,136.0,136.0,145.0,131.0,125.0
...,...,...,...,...,...,...
521605,16.0,20.0,14.0,9.0,4.0,NaN
521606,15.0,19.0,32.0,18.0,7.0,6.0
521607,34.0,33.0,31.0,29.0,28.0,46.0


In [ ]:
login_pivot.reset_index().head()

month,userid,02,03,04,05,06,07
0,1,7.0,7.0,6.0,9.0,6.0,15.0
1,2,86.0,82.0,71.0,84.0,84.0,80.0
2,3,130.0,133.0,118.0,122.0,126.0,141.0
3,4,85.0,68.0,65.0,66.0,71.0,71.0
4,5,151.0,136.0,136.0,145.0,131.0,125.0


In [ ]:
login_pivot.columns = ['02_login_times', '03_login_times', '04_login_times', '05_login_times', '06_login_times', '07_login_times']
login_pivot = login_pivot.fillna(0)

def month_slope(x):

    return np.mean([x["03_login_times"] - x["02_login_times"],
          x["04_login_times"] - x["03_login_times"],
          x["05_login_times"] - x["04_login_times"],
          x["06_login_times"] - x["05_login_times"],
          x["07_login_times"] - x["06_login_times"]
    ])


login_pivot["monthly_loging_slope"] = login_pivot.apply(month_slope, axis=1)

In [ ]:
login_pivot["login_times_sum"] = login_pivot[["02_login_times", "03_login_times", "04_login_times", "05_login_times", "06_login_times" , "07_login_times"]].sum(axis=1)
login_pivot["login_times_mean"] = login_pivot[["02_login_times", "03_login_times", "04_login_times", "05_login_times", "06_login_times" , "07_login_times"]].mean(axis=1)
login_pivot["login_times_min"] = login_pivot[["02_login_times", "03_login_times", "04_login_times", "05_login_times", "06_login_times" , "07_login_times"]].min(axis=1)
login_pivot["login_times_max"] = login_pivot[["02_login_times", "03_login_times", "04_login_times", "05_login_times", "06_login_times" , "07_login_times"]].max(axis=1)

In [ ]:
login_pivot.head()

,02_login_times,03_login_times,04_login_times,05_login_times,06_login_times,07_login_times,monthly_loging_slope,login_times_sum,login_times_mean,login_times_min,login_times_max
userid,,,,,,,,,,,
1,7.0,7.0,6.0,9.0,6.0,15.0,1.6,50.0,8.333333,6.0,15.0
2,86.0,82.0,71.0,84.0,84.0,80.0,-1.2,487.0,81.166667,71.0,86.0
3,130.0,133.0,118.0,122.0,126.0,141.0,2.2,770.0,128.333333,118.0,141.0
4,85.0,68.0,65.0,66.0,71.0,71.0,-2.8,426.0,71.000000,65.0,85.0
5,151.0,136.0,136.0,145.0,131.0,125.0,-5.2,824.0,137.333333,125.0,151.0


### purchase detail

In [ ]:
"""
By 月份user加總'category_encoded','total_amount','order_count'
"""
purchase_detail_final = purchase_detail.copy()
purchase_detail_final['month'] = purchase_detail_final['grass_date'].apply(lambda x: x[5:7])

In [ ]:
# 先算商品count數
purchase_detail_category = purchase_detail_final[["userid", "month", "category_encoded"]]
purchase_detail_category["category_encoded_count"] = purchase_detail_category["category_encoded"]
purchase_detail_category = purchase_detail_category.groupby(["userid", "month", "category_encoded"], as_index=False)["category_encoded_count"].count()

# join回去
purchase_detail_final = pd.merge(purchase_detail_final, purchase_detail_category, on=["userid", "month", "category_encoded"], how='left')

# 展開各月份商品by userid count數
purchase_detail_category = purchase_detail_final[["userid", "month", "category_encoded", "category_encoded_count"]]
purchase_detail_category["month_category_encoded"] = purchase_detail_category["month"] + "_" + purchase_detail_category["category_encoded"].astype(str)

purchase_detail_category = pd.pivot_table(purchase_detail_category, values="category_encoded_count", index=["userid"], columns=["month_category_encoded"]).fillna(0).reset_index()

purchase_detail_category = purchase_detail_category.rename_axis(None, axis=1)
purchase_detail_category.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,userid,02_1,02_10,02_11,02_12,02_13,02_14,02_15,02_16,02_17,02_18,02_19,02_2,02_20,02_21,02_22,02_23,02_3,02_4,02_5,02_6,02_7,02_8,02_9,03_1,03_10,03_11,03_12,03_13,03_14,03_15,03_16,03_17,03_18,03_19,03_2,03_20,03_21,03_22,03_23,...,06_15,06_16,06_17,06_18,06_19,06_2,06_20,06_21,06_22,06_23,06_3,06_4,06_5,06_6,06_7,06_8,06_9,07_1,07_10,07_11,07_12,07_13,07_14,07_15,07_16,07_17,07_18,07_19,07_2,07_20,07_21,07_22,07_23,07_3,07_4,07_5,07_6,07_7,07_8,07_9
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,2.0,1.0,1.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,2.0,1.0


In [ ]:
# 處理total_amount & order_count by month
melt_purchase = pd.melt(purchase_detail_final, id_vars=['userid','grass_date','month'], value_vars=['category_encoded','total_amount','order_count'])
melt_purchase = melt_purchase[~melt_purchase["variable"].eq("category_encoded")].reset_index(drop=True)
melt_purchase['new_var'] = melt_purchase['month'] + '_' + melt_purchase['variable']
melt_purchase = melt_purchase[["userid", "new_var", "value"]].groupby(["userid", "new_var"], as_index=False).sum()
purchase_detail_final = pd.pivot_table(data=melt_purchase, values='value', index='userid', columns='new_var').reset_index().fillna(0)
purchase_detail_final = purchase_detail_final.rename_axis(None, axis=1)

In [ ]:
# join all
purchase_detail_final = pd.merge(purchase_detail_final, purchase_detail_category, on="userid", how='left')
purchase_detail_final.isnull().sum()

userid             0
02_order_count     0
02_total_amount    0
03_order_count     0
03_total_amount    0
                  ..
07_5               0
07_6               0
07_7               0
07_8               0
07_9               0
Length: 151, dtype: int64

### user info

In [ ]:
# 去除不合理
# user_info = user_info[user_info["birth_year"] > 1920][user_info["birth_year"] < 2010].reset_index(drop=True)

In [ ]:
user_info["age"] = 2020 - user_info["birth_year"].values
user_info["enroll_days"] = 365 * 2021 - user_info["enroll_time"].str[:4].astype(int) * 365 + user_info["enroll_time"].str[5:7].astype(int) * 30 + user_info["enroll_time"].str[9:11].astype(int)
user_info = user_info[["userid", "gender", "is_seller", "age", "enroll_days"]].fillna(0)
user_info.head()

,userid,gender,is_seller,age,enroll_days
0,1,2.0,1,35.0,2347
1,2,1.0,1,40.0,2347
2,3,2.0,1,6.0,2347
3,4,1.0,1,42.0,2347
4,5,1.0,1,42.0,2347


In [ ]:
# def birthday_to_cat(birth):
#     """
#     把年齡分成5類
#     """
#     if birth <= 1920 or birth > 2010:
#         return 5
#     if birth <= 1980:
#         return 0
#     elif birth <= 1987:
#         return 1
#     elif birth <= 1992:
#         return 2
#     if birth <= 1997:
#         return 3
#     return 4

# user_info["birth_year"] = user_info["birth_year"].apply(birthday_to_cat)
# user_info.head()

,userid,gender,is_seller,birth_year,enroll_time
0,1,2.0,1,1,2015-05-27
1,2,1.0,1,0,2015-05-27
2,3,2.0,1,5,2015-05-27
3,4,1.0,1,0,2015-05-27
4,5,1.0,1,0,2015-05-27


In [ ]:
# def filter_old(enroll_time):
#     if enroll_time[2:4] in ["15", "16", "17", "18"]:
#         return 0
#     if enroll_time[2:4] in ["19"]:
#         return 1
#     return 2

# user_info["enroll_time"] = user_info["enroll_time"].apply(filter_old)
# user_info

,userid,gender,is_seller,birth_year,enroll_time
0,1,2.0,1,1,0
1,2,1.0,1,0,0
2,3,2.0,1,5,0
3,4,1.0,1,0,0
4,5,1.0,1,0,0
...,...,...,...,...,...
502152,521605,2.0,0,4,2
502153,521606,2.0,0,2,2
502154,521607,1.0,0,4,2
502155,521608,2.0,0,4,2


### merge data

In [ ]:
user_label_train_all = pd.merge(user_label_train, login_pivot, on='userid', how='left')
user_label_train_all = pd.merge(user_label_train_all, purchase_detail_final, on='userid', how='left')
user_label_train_all = pd.merge(user_label_train_all, user_info, on='userid', how='left')

In [ ]:
# user_label_train_all.to_csv("/content/drive/MyDrive/Colab Notebooks/2020_蝦皮BestCoder競賽(1121)/user_label_train_all.csv", index=None)

### LightGBM model

In [ ]:
list(user_label_train_all.columns)

['userid',
 'label',
 '02_login_times',
 '03_login_times',
 '04_login_times',
 '05_login_times',
 '06_login_times',
 '07_login_times',
 'monthly_loging_slope',
 'login_times_sum',
 'login_times_mean',
 'login_times_min',
 'login_times_max',
 '02_order_count',
 '02_total_amount',
 '03_order_count',
 '03_total_amount',
 '04_order_count',
 '04_total_amount',
 '05_order_count',
 '05_total_amount',
 '06_order_count',
 '06_total_amount',
 '07_order_count',
 '07_total_amount',
 '02_1',
 '02_10',
 '02_11',
 '02_12',
 '02_13',
 '02_14',
 '02_15',
 '02_16',
 '02_17',
 '02_18',
 '02_19',
 '02_2',
 '02_20',
 '02_21',
 '02_22',
 '02_23',
 '02_3',
 '02_4',
 '02_5',
 '02_6',
 '02_7',
 '02_8',
 '02_9',
 '03_1',
 '03_10',
 '03_11',
 '03_12',
 '03_13',
 '03_14',
 '03_15',
 '03_16',
 '03_17',
 '03_18',
 '03_19',
 '03_2',
 '03_20',
 '03_21',
 '03_22',
 '03_23',
 '03_3',
 '03_4',
 '03_5',
 '03_6',
 '03_7',
 '03_8',
 '03_9',
 '04_1',
 '04_10',
 '04_11',
 '04_12',
 '04_13',
 '04_14',
 '04_15',
 '04_16',
 '04

In [ ]:
X = user_label_train_all.loc[:, ~user_label_train_all.columns.isin(["label"])]
y = user_label_train_all["label"]

In [ ]:
cat_colnames = ["gender", "is_seller"]

In [ ]:
for col in cat_colnames:
    try:
        X[col] = X[col].astype('category')
    except Exception:
        print("No ", col, " exists")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### CV

In [ ]:
# Seed = 111
sss = StratifiedKFold(n_splits=4, random_state=111, shuffle=False)

for train_index, test_index in sss.split(X, y):
    print("Train:", train_index, "Test:", test_index)
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Train: [106663 106665 106666 ... 426829 426830 426831] Test: [     0      1      2 ... 106822 106823 106825]
Train: [     0      1      2 ... 426829 426830 426831] Test: [106663 106665 106666 ... 214299 214303 214306]
Train: [     0      1      2 ... 426829 426830 426831] Test: [213014 213017 213018 ... 320964 320968 320969]
Train: [     0      1      2 ... 320964 320968 320969] Test: [319707 319712 319713 ... 426829 426830 426831]


In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(320124, 166)
(320124,)
(106708, 166)
(106708,)


### Run

In [ ]:
model_gbdt = LGBMClassifier(objective='binary',
                             boosting_type='gbdt',
                             num_leaves=15,  # < 2^max_depth
                             # max_depth=6,
                             reg_lambda=0.5,
                             learning_rate=0.01,
                             n_estimators=2200,
                             # max_bin=200, # default=255
                             bagging_fraction=0.8,
                             # min_child_weight=200,
                             # bagging_freq=5, 
                             # bagging_seed=7,
                             # min_data_in_leaf = 19,
                             feature_fraction=0.8,
                             scale_pos_weight=1.5,
                             random_state=111,
                             # scale_pos_weight: for high class imbalance learning task parameters
                             # sample_pos_weight = number of negative samples / number of positive samples
                             # is_training_metric=True
                             # is_unbalance=True
                             # early_stopping_rounds=100
                             # feature_fraction_seed=7
                         )

# lgb_param = {
#     'boosting_type': 'gbdt',
#     'objective': 'regression',
#     'learning_rate': 0.012783478569491262,
#     'num_leaves': 303,
#     'max_depth': 23,
#     'min_data_in_leaf': 19,
#     'subsample': 0.8499384738625326,
#     'colsample_bytree': 0.3838563405503983,
#     'reg_alpha': np.exp(-1.6155277167195283),
#     'reg_lambda': np.exp(-0.7142681482849973),
#     'n_estimators': int(np.exp(9.82895782295262)),
#     'n_jobs': 5,
# } 

# model_dart = LGBMClassifier(objective='binary', 
#                              boosting_type='dart',
#                              num_leaves=31,  # < 2^max_depth
#                              reg_lambda=0.5,
#                              learning_rate=0.01,
#                              n_estimators=10000,  # 30000
#                              # max_bin=200, # default=255
#                              bagging_fraction=0.8,
#                              scale_pos_weight=1.2,
#                              # bagging_freq=5, 
#                              # bagging_seed=7,
#                              feature_fraction=0.8,
#                              # drop_rate=0.3,  # default=0.1
#                              max_drop=10,  # default=50, 每iteration最多丟掉幾顆樹
#                              skip_drop=0.5,  # default=0.5 每iteration決定dropout的機率
#                              # xgboost_dart_mode=True,  # default=False, 是否要用xgb的dart模式
#                              # uniform_drop=True,  # set this to true, if you want to use uniform drop
#                              # drop_seed=11,  # random seed to choose dropping models
#                              # early_stopping_rounds=100
#                              # feature_fraction_seed=7
#                          )

In [ ]:
np.exp(-1.6155277167195283)
np.exp(-0.7142681482849973)
int(np.exp(9.82895782295262))

18563

In [ ]:
result = model_gbdt.fit(X, y, #X_train, y_train,
              # eval_set=[(X_train, y_train)],
              # eval_set=[(X_val, y_val)],
              eval_metric='auc',
              # eval_metric="poisson",
              # eval_metric=lgb_f1_score,
              # categorical_feature=categorical_feature_idx,
              # early_stopping_rounds=30,
              verbose=1
             )

# result = model_dart.fit(X_train, y_train,
#               # eval_set=[(X_train, y_train)],
#               eval_set=[(X_val, y_val)],
#               eval_metric='auc',
#               # eval_metric="poisson",
#               # eval_metric=lgb_f1_score,
#               # categorical_feature=categorical_feature_idx,
#               early_stopping_rounds=30,
#               verbose=1
#              )

## Submission

In [ ]:
submission = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/2020_蝦皮BestCoder競賽(1121)/iamthebestcoderopen2020/submission.csv", index_col=None)

In [ ]:
submission_all = pd.merge(submission, login_pivot, on='userid', how='left')
submission_all = pd.merge(submission_all, purchase_detail_final, on='userid', how='left')
submission_all = pd.merge(submission_all, user_info, on='userid', how='left')

# 
submission_all_X = submission_all.loc[:, list(X.columns)]

cat_colnames = ["gender", "is_seller"]

for col in cat_colnames:
    try:
        submission_all_X[col] = submission_all_X[col].astype('category')
    except Exception:
        print("No ", col, " exists")

pred = model_gbdt.predict_proba(submission_all_X)
submission_file = pd.concat([submission, pd.Series(pred[:, 1])], axis=1)
submission_file.columns = ["userid", "label"]
submission_file.to_csv("/content/drive/MyDrive/Colab Notebooks/2020_蝦皮BestCoder競賽(1121)/late_submission_baseline_re-eng_v5_gdbt_AllData_prob.csv", index=None)